<a href="https://colab.research.google.com/github/ppkgtmm/emotion/blob/main/emo_gather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
data_dir = '/content/drive/MyDrive/emotion/data'
data_files = ['goemotions_1.csv','goemotions_2.csv', 'goemotions_3.csv']

In [4]:
df = pd.DataFrame({})

In [7]:
for data_file in data_files:
  temp = pd.read_csv(f"{data_dir}/{data_file}")
  df = pd.concat([df,temp])

In [12]:
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [15]:
df.columns

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [16]:
unrelated_cols = ['id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear']

In [17]:
data = df.drop(columns=unrelated_cols)

In [18]:
data.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [19]:
data = data.melt(id_vars='text', value_vars=data.columns[1:])

In [45]:
no_unuseful = data[data['value'] == 1]
no_unuseful.head()

,text,variable,value
15,"I appreciate it, that's good to know. I hope I...",admiration,1
18,Pretty much every Punjabi dude I've met.,admiration,1
28,"Lots, play store or apple store vpn. Nord is good",admiration,1
35,nice!! I'll try this one,admiration,1
50,She’s like a kewpie doll with them. Precious.,admiration,1


In [46]:
no_unuseful = no_unuseful.drop(columns=['value'])

In [47]:
no_unuseful.shape

(332174, 2)

In [48]:
no_unuseful.columns = ['text','target']
no_unuseful.head()

,text,target
15,"I appreciate it, that's good to know. I hope I...",admiration
18,Pretty much every Punjabi dude I've met.,admiration
28,"Lots, play store or apple store vpn. Nord is good",admiration
35,nice!! I'll try this one,admiration
50,She’s like a kewpie doll with them. Precious.,admiration


In [49]:
no_unuseful = no_unuseful.drop_duplicates(subset=['text','target'], keep='last')

In [50]:
no_unuseful[['text','target']].describe()

,text,target
count,164954,164954
unique,57730,28
top,"Welcome to racial integration, where your civi...",neutral
freq,13,31446


In [51]:
duplicates = pd.concat(g for _, g in no_unuseful.groupby("text") if len(g) > 1)

In [54]:
duplicates.groupby('text').nunique().reset_index() \
.sort_values('target').head()

,text,target
24675,My boy [NAME] <3,2
16218,I think [NAME] has a pretty unconventional fac...,2
16217,I think [NAME] does a great job! Can't believe...,2
34640,The best ad placement.,2
34642,The best album.,2


In [55]:
no_unuseful.to_csv('/content/drive/MyDrive/emotion/data/goemotions.csv')